In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../data/raw/melb_data.csv' , usecols=['Rooms', 'Type', 'Price', 'Method', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Propertycount'])
df.Landsize = (df.Landsize).replace(0, np.nan)
df.BuildingArea = (df.BuildingArea).replace(0, np.nan)
df.head()

,Rooms,Type,Price,Method,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Propertycount
0,2,h,1480000.0,S,2.5,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,4019.0
1,2,h,1035000.0,S,2.5,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,4019.0
2,3,h,1465000.0,SP,2.5,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,4019.0
3,3,h,850000.0,PI,2.5,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,4019.0
4,4,h,1600000.0,VB,2.5,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,4019.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rooms          13580 non-null  int64  
 1   Type           13580 non-null  object 
 2   Price          13580 non-null  float64
 3   Method         13580 non-null  object 
 4   Distance       13580 non-null  float64
 5   Bedroom2       13580 non-null  float64
 6   Bathroom       13580 non-null  float64
 7   Car            13518 non-null  float64
 8   Landsize       11641 non-null  float64
 9   BuildingArea   7113 non-null   float64
 10  YearBuilt      8205 non-null   float64
 11  CouncilArea    12211 non-null  object 
 12  Propertycount  13580 non-null  float64
dtypes: float64(9), int64(1), object(3)
memory usage: 1.3+ MB


In [5]:
df.isna().sum()

Rooms               0
Type                0
Price               0
Method              0
Distance            0
Bedroom2            0
Bathroom            0
Car                62
Landsize         1939
BuildingArea     6467
YearBuilt        5375
CouncilArea      1369
Propertycount       0
dtype: int64

---
# 6 Different ways to Handling Missing Values

<h2>1. Do Nothing</h2>
<ul>
    <li>We just let the algorithm handle the missing data
    <li>This only works for several algorithm (ie. XGBoost and LightGBM)
    <li>XGBoost can factor in the missing values and learn the best imputation values for the missing data based on the training loss reduction
    <li>LightGBM has the option to just ignore them (<i>use_missing=false</i>)
    <li>However, other algorithms will panic and throw an error complaining about the missing values (ie. Scikit learn — LinearRegression) so we need to handle the missing data and clean it before feeding it to the algorithm
</ul>

---
<h2>2. Remove rows</h2>
<ul>
    <li>The simplest approach for dealing with missing values is to remove entire predictor(s) and/or sample(s) that contain missing values
    <li>If we want to to this, please make sure we just remove maximum 3% from our total data observations
</ul>

In [6]:
df_test = df.copy()
df_test.dropna(subset=['Car'], axis=0, inplace=True)
total = ((df.shape[0] - df_test.shape[0])/df.shape[0]) * 100
print('original observations : {}\nafter remove Null values : {}\nTotal observations were removed : {:.4f}%'.format(df.shape[0], df_test.shape[0], total))

original observations : 13580
after remove Null values : 13518
Total observations were removed : 0.4566%


---
<h2>3. Imputation using Mean/Median values</h2>
<strong>Pros</strong><ul>
    <li>Easy and fast
    <li>Works well with small numerical datasets
</ul>
<strong>Cons</strong><ul>
    <li>Doesn’t factor the correlations between features, it only works on the column level
    <li>Will give poor results on encoded categorical features
    <li>Not very accurate
    <li>Doesn’t account for the uncertainty in the imputations
</ul>

In [7]:
from sklearn.impute import SimpleImputer

df_test = df.copy()
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer.fit(df_test[['Car']])
df_test[['Car']] = imputer.transform(df_test[['Car']])
df_test.Car.describe(include='all')

count    13580.000000
mean         1.611856
std          0.960793
min          0.000000
25%          1.000000
50%          2.000000
75%          2.000000
max         10.000000
Name: Car, dtype: float64

---
<h2>4. Imputation using Most Frequent or Zero/Constant values</h2>
<strong>Pros</strong><ul>
    <li>Works well with categorical features
</ul>
<strong>Cons</strong><ul>
    <li>Doesn’t factor the correlations between features, it only works on the column level
    <li>It can introduce bias in the data
</ul>

In [8]:
from sklearn.impute import SimpleImputer

df_test = df.copy()
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(df_test[['Car']])
df_test[['Car']] = imputer.transform(df_test[['Car']])
df_test.Car.describe(include='all')

count    13580.000000
mean         1.611856
std          0.960793
min          0.000000
25%          1.000000
50%          2.000000
75%          2.000000
max         10.000000
Name: Car, dtype: float64

---
<h2>5. Imputation using k-NN</h2>
<strong>Pros</strong><ul>
    <li>Can be much more accurate than the mean, median or most frequent imputation methods (It depends on the dataset)
</ul>
<strong>Cons</strong><ul>
    <li>Computationally expensive because KNN works by storing the whole training dataset in memory
    <li>K-NN is quite sensitive to outliers in the data (unlike SVM)
</ul>
<strong>How it works</strong><ul>
    <li>It creates a basic mean impute then uses the resulting complete list to construct a KDTree
    <li>Then, it uses the resulting KDTree to compute nearest neighbours (NN)
    <li>After it finds the k-NNs, it takes the weighted average of them
</ul>

In [9]:
from sklearn.impute import KNNImputer

df_test = df.copy()
imputer = KNNImputer(missing_values=np.nan, n_neighbors=5)
imputer.fit(df_test[['Car']])
df_test[['Car']] = imputer.transform(df_test[['Car']])
df_test.Car.describe(include='all')

count    13580.000000
mean         1.610075
std          0.960433
min          0.000000
25%          1.000000
50%          2.000000
75%          2.000000
max         10.000000
Name: Car, dtype: float64

---
<h2>6. Imputation using <i>Multivariate Imputation by Chained Equation</i> (MICE)</h2>
<strong>How it works</strong><ul>
    <li>This type of imputation works by filling the missing data multiple times
    <li>Multiple Imputations (MIs) are much better than a single imputation as it measures the uncertainty of the missing values in a better way
    <li>The chained equations approach is also very flexible and can handle different variables of different data types (ie., continuous or binary) as well as complexities such as bounds or survey skip patterns
</ul>

In [10]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

df_test = df.copy()
imputer = IterativeImputer(missing_values=np.nan, max_iter=10, random_state=0)
imputer.fit(df_test[['Car']])
df_test[['Car']] = imputer.transform(df_test[['Car']])
df_test.Car.describe(include='all')

count    13580.000000
mean         1.610075
std          0.960433
min          0.000000
25%          1.000000
50%          2.000000
75%          2.000000
max         10.000000
Name: Car, dtype: float64